This is a example of two dof Duffing oscillator based on the paper:

[A SIMPLE CRITERION FOR ESTABLISHING AN UPPER LIMIT TO THE HARMONIC EXCITATION LEVEL OF THE DUFFING OSCILLATOR USING THE VOLTERRA SERIES](https://www.sciencedirect.com/science/article/pii/S0022460X96900917)

The following nonlinear equation of motion is solve:

$$
\begin{align}
m_1 \ddot{x}_1 + 2 c \dot{x}_1 - c \dot{x}_2 + 2 k_1 x_1 - k_1 x_2 - \beta(x_2 - x_1)^3   & = P cos( \omega t) \\
m_2 \ddot{x}_2 + 2 c \dot{x}_2 - c \dot{x}_1 + 2 k_1 x_2 - k_1 x_1 + \beta(x_2 - x_1)^3   & = 0
\end{align}
$$

Where $m_1 = m_2 = 1 kg$, $c = 0.05 Ns/m $  , $k_1 = 1.0 N/m$ and $\beta = 5.0 N/m^3$ 

Rewritten the two above equation in matrix notation we get:

\begin{equation*}
\begin{bmatrix}
m_1 & 0 \\
0 & m_2 
\end{bmatrix}
\begin{bmatrix}
\ddot{x}_1 \\
\ddot{x}_2
\end{bmatrix}
+
\begin{bmatrix}
2c & -c \\
-c & 2c 
\end{bmatrix}
\begin{bmatrix}
\dot{x}_1 \\
\dot{x}_2
\end{bmatrix}
+
\begin{bmatrix}
2k_1 & -k_1 \\
-k_1 & 2k_1 
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
+
\begin{bmatrix}
f^{NL}_1(x_1,x_2) \\
f^{NL}_2(x_1,x_2)
\end{bmatrix}
=
\begin{bmatrix}
P cos( \omega t) \\
0
\end{bmatrix}
\end{equation*}

Where $f^{NL}_1(x_1,x_2)= -\beta (x_2 - x_1)^3$ and $f^{NL}_2(x_1,x_2)= \beta (x_2 - x_1)^3$ 

In compact matrix notation notation we have:

$$
M \ddot{x} + C \dot{x} + K x + f^{NL}(x) = f^L
$$

Where $f^L$ is the vector function $ [ P cos( \omega t), 0 ]^T $ and $f^{NL}(x)$ is defined as:

$$
f^{NL}(x) = \beta H  (B^{\Delta} x)^3
$$ 

where $H$ is the matrix:
\begin{equation*}
H = 
\begin{bmatrix}
-1 & 0 \\
0 & 1
\end{bmatrix}
\end{equation*}

and $B^{\Delta}$ is 

\begin{equation*}
B^{\Delta} = 
\begin{bmatrix}
-1 & 1 \\
-1 & 1
\end{bmatrix}
\end{equation*}


If we apply a Fourier Transform $\mathscr{F}$ in the above system we have:
$$
Z( \omega ) \tilde{x} + \tilde{f}^{NL} (\tilde{x}) - \tilde{f}^L = 0
$$

where 
$$
Z( \omega ) = -\omega^2 M + j \omega C + K
$$

$$
\tilde{f}^L = [P,0]^T
$$

$$
\tilde{f}^{NL}(\tilde{x}) = \mathscr{F}(f^{NL}(\mathscr{F}^{-1}(\tilde{x})))
$$

In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from contpy import frequency
from contpy import optimize
from contpy.cases.case1 import name, n_dofs, K,M,C,P,beta, B_delta, H, Tc


In [2]:
#HBM variables
nH = 2
n_points = 2000


Q = frequency.assemble_HBMOperator(n_dofs,number_of_harm=nH ,n_points=n_points)
P_aug = list(0*P)*nH
P_aug[0:n_dofs] = list(P)
P_aug = np.array(P_aug)
fl = Q.dot(P_aug).real


fl_ = Q.H.dot(fl) # force in frequency domain
fnl = lambda u : beta*(Tc.dot(u)**3)
fnl_ = lambda u_ : Q.H.dot(fnl(Q.dot(u_))) - fl_
Z = lambda w : frequency.create_Z_matrix(K,C,M,f0= w/(2.0*np.pi),nH=nH, static=False)
R = lambda u_, w : Z(w).dot(u_) + fnl_(u_)


In [3]:
%%time
x0 = np.array([0.0]*n_dofs*nH,dtype=np.complex)
y_d, p_d, info_dict = optimize.continuation(R,x0=x0,p_range=(0.01,3.0), p0=0.1, correction_method='matcont',
                                            max_int=2000, max_dp=0.05,step=0.1, max_int_corr=30, tol=1.0E-10)


Continuation algorithm has reached the limits of parameter range
Wall time: 9min 46s


In [4]:
fig, ax1 = plt.subplots(1)
max_x = np.abs(p_d).max()
min_x = np.abs(p_d).min()
max_y = np.abs(y_d[:]).max()
min_y = np.abs(y_d[:]).min()

factor = 0.1
lim = lambda x, factor  : x + np.sign(x)*factor*x

ax1.set_xlim([lim(min_x,-factor)   ,lim(max_x,factor)])
ax1.set_ylim([lim(min_y,-factor)   ,lim(max_y,factor)])


def update(i):
    ax1.clear()
    ax1.plot(p_d[i],np.abs(y_d[0][i]),'ro',markerfacecolor='y')
    ax1.plot(p_d[:i],np.abs(y_d[0][:i]),'k.',label='dof 1')
    ax1.plot(p_d[i],np.abs(y_d[1][i]),'bo',markerfacecolor='y')
    ax1.plot(p_d[:i],np.abs(y_d[1][:i]),'b.', label='dof 2')
    ax1.set_xlim([lim(min_x,-factor)   ,lim(max_x,factor)])
    ax1.set_ylim([lim(min_y,-factor)   ,lim(max_y,factor)])
    ax1.set_xlabel('Frequency [rad/s]', fontsize=12)
    ax1.set_ylabel('Normalized Amplitude', fontsize=12)
    ax1.set_title('%s with %i Harmonics and %i time samples' %(name,nH,n_points), fontsize=12)
    ax1.legend()
    
#ani = FuncAnimation(fig, update, frames=np.arange(0, len(p_d) ,1 ), blit=True, interval=1)   
update(len(p_d)-1)

if False:
    Writer = animation.writers['imagemagick']
    writer = Writer(fps=30, metadata=dict(artist='Guilherme Jenovencio'), bitrate=1800)
    ani.save('2dof_couple_duffing_ocsillator_v1.gif', dpi=200, writer=writer)

<IPython.core.display.Javascript object>

C:\Users\ge72tih\AppData\Local\Continuum\anaconda3\envs\ndcsi\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
